In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas

In [2]:
def plt2frame(fig):
    canvas = FigureCanvas(fig)
    canvas.draw()
    frame = np.frombuffer(canvas.buffer_rgba(), dtype='uint8')
    frame = frame.reshape(fig.canvas.get_width_height()[::-1] + (4,))
    frame = cv2.cvtColor(frame, cv2.COLOR_RGBA2RGB)
    plt.close(fig)

    return frame


def getSparsityPlot(messages, videoOut = True):

    #stdMessage = np.std(messages,axis = -1)
    stdMessage = np.std(messages,axis = 0)
    
    fig, ax = plt.subplots(1, 1)
    ax.pcolormesh(stdMessage[np.argsort(stdMessage)[::-1][None, :15]], cmap='gray_r', edgecolors='k')
    plt.axis('off')
    plt.grid(True)
    ax.set_aspect('equal')
    plt.text(15.5, 0.5, '...', fontsize=30)
    plt.tight_layout()

    if videoOut:
        return plt2frame(fig)
    
    else:
        return None

In [3]:
def create_mp4_with_frames(output_path, frames, fps=10):
    if not frames:
        raise ValueError("No frames to write to video.")
    
    height, width, _ = frames[0].shape
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video_writer = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    for frame in frames:
        video_writer.write(frame)
    
    video_writer.release()

def combine_video_with_new_frame(input_path, output_path, new_frame):
    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        print(f"Error: Cannot open video file {input_path}")
        return

    frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)
    
    cap.release()
    frames.append(new_frame)
    create_mp4_with_frames(output_path, frames)

In [4]:
name = '/master/code/experimental/new_plots_functions'
import os

os.path.exists(name)

True

In [5]:
name = '/master/code/experimental/new_plots_functions/test-message-0.mp4'

for i in range(20):
    a = np.random.random((1000, 128))
    frame = getSparsityPlot(a)

    if i == 0:
        create_mp4_with_frames(name, [frame])
    else:
        combine_video_with_new_frame(name, name, frame)